In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from tqdm import tqdm
# from pandarallel import pandarallel

from matplotlib import pyplot as plt
from tqdm.contrib.concurrent import process_map

# pandarallel.initialize(progress_bar=True)
sns.set_theme()

In [3]:
# !pip install 

In [4]:
def get_samples(files):
    dfs = process_map(pd.read_csv, files, max_workers=24, chunksize=100)
    df = pd.concat(dfs, ignore_index=True)
    return df, dfs

In [5]:
train_path = Path("../data/train")
test_path = Path("../data/test")

train_files = list(train_path.glob("**/*.csv"))
test_files = list(test_path.glob("*.csv"))

train_df, train_dfs = get_samples(train_files)
test_df, test_dfs = get_samples(test_files)

  0%|          | 0/832798 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
save_columns = ['roll', 'aEgo', 'vEgo', 'latAccelSteeringAngle', 'steeringAngleDeg']

for i in range(len(train_dfs)):
    train_dfs[i] = train_dfs[i][save_columns].rename(columns={
        'latAccelSteeringAngle': 'targetLateralAcceleration',
        'steeringAngleDeg': 'steerCommand'
    })
    
train_df = train_df[save_columns].rename(columns={
    'latAccelSteeringAngle': 'targetLateralAcceleration',
    'steeringAngleDeg': 'steerCommand'
})

In [5]:
scalers = {
    'aEgo': StandardScaler(),
    'vEgo': StandardScaler(),
    'roll': StandardScaler(),
    'targetLateralAcceleration': StandardScaler() # RobustScaler() 
}

def scale_steering_by_first_10_seconds(df, steering_col='steerCommand'):
    first_10 = df.iloc[:100]
    scaler = RobustScaler()
    scaler.fit(first_10[[steering_col]])
    df_scaled = df.copy()
    df_scaled[steering_col] = scaler.transform(df[[steering_col]])
    return df_scaled

for col, scaler in tqdm(scalers.items(), desc="Fitting scalers"):
    scaler.fit(train_df[[col]])
    
train_scaled = []
for df in tqdm(train_dfs, desc="Scaling training data"):
    df_scaled = df.copy()
    for col, scaler in scalers.items():
        df_scaled[col] = scaler.transform(df[[col]])
    df_scaled = scale_steering_by_first_10_seconds(df_scaled)
    train_scaled.append(df_scaled)

test_scaled = []
for df in tqdm(test_dfs):
    df_scaled = df.copy()
    for col, scaler in scalers.items():
        df_scaled[col] = scaler.transform(df[[col]])
    df_scaled = scale_steering_by_first_10_seconds(df_scaled)
    test_scaled.append(df_scaled)

In [6]:
train_scaled[0].head()

,roll,aEgo,vEgo,targetLateralAcceleration,steerCommand
0,-0.795990,0.255430,-0.217987,0.198525,-0.187500
1,-0.830347,0.283653,-0.216903,0.199554,-0.187500
2,-0.864705,0.288052,-0.215774,0.144675,-0.062679
3,-0.899062,0.268569,-0.215247,0.095632,0.062037
4,-0.924962,0.694610,-0.211312,0.072798,0.124887


In [7]:
test_scaled[0].head()

,t,vEgo,aEgo,roll,targetLateralAcceleration,steerCommand
0,0.0,1.598870,0.067468,0.596711,-0.076423,-0.439803
1,0.1,1.598888,0.012449,0.594146,-0.025446,-0.523350
2,0.2,1.598994,-0.019195,0.591581,0.009397,-0.471967
3,0.3,1.599457,0.067695,0.586913,0.008378,-0.496606
4,0.4,1.600724,0.216295,0.580619,0.021605,-0.649739


In [8]:
# !pip3 install torch torchvision torchaudio


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import pickle

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Model ---
class LstmEncoderDecoder(nn.Module):
    def __init__(self, physics_input_size, control_input_size, hidden_size, num_layers, dropout=0.2):
        super().__init__()
        self.physics_encoder = nn.LSTM(physics_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.control_encoder = nn.LSTM(control_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.decoder = nn.LSTM(control_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_size, 1)  # Predict targetLateralAcceleration

    def forward(self, input_physics, input_control_sequence):
        _, (hidden_phsc, cell_phsc) = self.physics_encoder(input_physics)
        _, (hidden_ctrl, cell_ctrl) = self.control_encoder(input_control_sequence)
        
        hidden_enc = (hidden_phsc + hidden_ctrl) / 2
        cell_enc = (cell_phsc + cell_ctrl) / 2
        
        decoder_output, _ = self.decoder(input_control_sequence, (hidden_enc, cell_enc))
        
        output = self.fc_out(decoder_output)
        return output

In [22]:
class DrivingDataset(Dataset):
    def __init__(self, dfs, seq_len=20):
        self.samples = []
        for df in dfs:
            arr = df[['roll', 'aEgo', 'vEgo', 'targetLateralAcceleration', 'steerCommand']].values
            for i in range(len(arr) - seq_len - 1):
                physics_input = arr[i:i+seq_len, :3]
                control_input = arr[i:i+seq_len, 3:]
                y = arr[i+1:i+seq_len+1, 4]    # target: next steerCommand
                self.samples.append((physics_input, control_input, y.reshape(-1, 1)))
                
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        physics_input, control_input, y = self.samples[idx]
        return (torch.tensor(physics_input, dtype=torch.float32),
                torch.tensor(control_input, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32))

def train_val_split(dfs, val_ratio=0.2, seed=42):
    """Split list of dataframes into training and validation sets"""
    random.seed(seed)
    n_val = int(len(dfs) * val_ratio)
    
    # Shuffle the indices
    indices = list(range(len(dfs)))
    random.shuffle(indices)
    
    # Split into train and validation
    val_indices = indices[:n_val]
    train_indices = indices[n_val:]
    
    train_dfs = [dfs[i] for i in train_indices]
    val_dfs = [dfs[i] for i in val_indices]
    
    return train_dfs, val_dfs


In [23]:
def train_model(train_dfs, val_dfs=None, num_epochs=5, batch_size=64, seq_len=20,
                lr=1e-3, hidden_size=128, num_layers=4):
    train_dataset = DrivingDataset(train_dfs, seq_len=seq_len)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    if val_dfs:
        val_dataset = DrivingDataset(val_dfs, seq_len=seq_len)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    model = LstmEncoderDecoder(
        physics_input_size=3,
        control_input_size=2,
        hidden_size=hidden_size,
        num_layers=num_layers
    ).to(DEVICE)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        
        for physics_input, control_input, y in tqdm(train_loader):
            physics_input = physics_input.to(DEVICE)
            control_input = control_input.to(DEVICE)
            y = y.to(DEVICE)
            
            optimizer.zero_grad()
            out = model(physics_input, control_input)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_loader)
        
        if val_dfs:
            model.eval()
            total_val_loss = 0
            
            with torch.no_grad():
                for physics_input, control_input, y in val_loader:
                    physics_input = physics_input.to(DEVICE)
                    control_input = control_input.to(DEVICE)
                    y = y.to(DEVICE)
                    
                    out = model(physics_input, control_input)
                    loss = criterion(out, y)
                    total_val_loss += loss.item()
            
            avg_val_loss = total_val_loss / len(val_loader)
            
            # Save best model
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), "../models/lstm_best_model.pt")

            print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        else:
            print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f}")
    
    return model

In [24]:
train_split, val_split = train_val_split(train_scaled)
model_version = 'base_v1_full_dataset_10_epochs'
model = train_model(train_split, val_split, num_epochs=10)


100%|██████████| 25309/25309 [02:22<00:00, 178.16it/s]


Epoch 1/10 | Train Loss: 71.1661 | Val Loss: 694.7622


100%|██████████| 25309/25309 [02:26<00:00, 172.44it/s]


Epoch 2/10 | Train Loss: 34.0143 | Val Loss: 616.6923


100%|██████████| 25309/25309 [02:21<00:00, 178.61it/s]


...

Epoch 8/10 | Train Loss: 19.1502 | Val Loss: 489.5398


100%|██████████| 25309/25309 [02:21<00:00, 178.69it/s]


Epoch 9/10 | Train Loss: 17.5664 | Val Loss: 502.3697


100%|██████████| 25309/25309 [02:25<00:00, 173.46it/s]


Epoch 10/10 | Train Loss: 14.4830 | Val Loss: 474.0756


In [28]:
torch.save(model.state_dict(), "../models//v1/lstm_lataccel.pt")
for name, scaler in scalers.items():
    if hasattr(scaler, 'feature_names_in_'):
        scaler.feature_names_in_ = None

with open("../models/scalers.pkl", "wb") as f:
    pickle.dump(scalers, f)